In [1]:
! pip install rank_bm25

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#!pip install rank_bm25
! git clone https://github.com/cr-nlp/project1-2023.git

import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import regex
from collections import defaultdict
import nltk
nltk.download('stopwords')
nltk.download('punkt')


def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    special_char_pattern = r'[^a-zA-Z0-9\s]+'
    
    # Use re.sub to replace special characters with spaces
    cleaned_string = regex.sub(special_char_pattern, ' ', val)
    
    return cleaned_string

def text2TokenList(text):
	
	stopword = stopwords.words('english')

	word_tokens = word_tokenize(text)
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops



fatal: destination path 'project1-2023' already exists and is not an empty directory.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siney\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siney\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
dicDoc, dicReq, dicReqDoc = loadNFCorpus()

In [4]:
for req in dicReq: 
    dicReq[req] = clean_sentence(dicReq[req])

In [5]:
for doc in dicDoc: 
    dicDoc[doc] = clean_sentence(dicDoc[doc])

### Using bm25 to get for each query , the top 20 documents and then we will use word2vec

In [6]:
from rank_bm25 import BM25Okapi

In [7]:
doc_ids_to_keep={}
for reqid in dicReq:
    if len(dicReqDoc[reqid]) != 0:
    
        token_req = text2TokenList(dicReq[reqid])
        corpus_doc_req = [i for i in dicReqDoc[reqid]]

        tokens_docs={}
        for docid in corpus_doc_req:
            token_doc = text2TokenList(dicDoc[docid])
            tokens_docs[docid] = token_doc
        
        bm25 = BM25Okapi(tokens_docs.values())
        doc_score = bm25.get_scores(text2TokenList(dicReq[reqid]))
        docToKeep= bm25.get_top_n(text2TokenList(dicReq[reqid]), list(tokens_docs.values()), n=20)
    
        doc_ids_to_keep[reqid] = [key for key in tokens_docs.keys() if tokens_docs[key] in docToKeep]
    else:
        continue

## Creation of the corporas and the word2vec

In [8]:
def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for sentence in data:
            word_list = sentence.strip(' \n')
            word_list = text2TokenList(word_list)
            corpus.append(word_list)

    return corpus


In [9]:
reqTokeep = {}
for reqid in doc_ids_to_keep:
    reqTokeep[reqid] = dicReq[reqid]
corpus_req = build_corpus(reqTokeep.values())

In [10]:
dicDoctokeep = {}
for list in doc_ids_to_keep.values():
    for key in list : 
        dicDoctokeep[key] =dicDoc[key]


In [12]:
import gensim
model2 = "PubMed-w2v.bin" 
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(model2, binary=True)

In [13]:
model_doc = word_vectors

In [14]:
def get_vector(text):
    try:
        vector =model_doc.get_vector(text) 
        return np.mean(vector)
    except KeyError:
        #print(f"{text} not found.")
        return 0

In [15]:
from rank_bm25 import BM25Okapi

corpusDocTokenList = []
corpusReqTokenList = {}
corpusDocName=[]
corpusDicoDocName={}
i = 0
for k in dicDoctokeep:
  docTokenList = [get_vector(i) for i in text2TokenList(dicDoc[k])]
  corpusDocTokenList.append(docTokenList)
  corpusDocName.append(k)
  corpusDicoDocName[k] = i
  i = i + 1

print("reqs...")
corpusReqName=[]
corpusDicoReqName={}
i = 0
for k in reqTokeep:
  reqTokenList = [get_vector(i) for i in text2TokenList(dicReq[k])]
  corpusReqTokenList[k] = reqTokenList
  corpusReqName.append(k)
  corpusDicoReqName[k] = i
  i = i + 1

reqs...


In [16]:
#print("bm25 doc indexing...")
bm25 = BM25Okapi(corpusDocTokenList)

ndcgCumul=0
corpusReqVec={}
ndcgBM25Cumul=0
nbReq=0

from sklearn.metrics import ndcg_score

for req in corpusReqTokenList:
  j=0
  reqTokenList = corpusReqTokenList[req]
  doc_scores = bm25.get_scores(reqTokenList)
  trueDocs = np.zeros(len(corpusDocTokenList))

  for docId in corpusDicoDocName:
    if req in reqTokeep.keys():
      if docId in doc_ids_to_keep[req]:
        #get position docId
        posDocId = corpusDicoDocName[docId]
        trueDocs[posDocId] = dicReqDoc[req][docId]
        
        #print("TOKEEP=",docId)
        #print(trueDocs)
  
  ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=5)
  nbReq = nbReq + 1
ndcgBM25Cumul = ndcgBM25Cumul / nbReq
print("ndcg bm25=",ndcgBM25Cumul)


ndcg bm25= 0.41212531762160715
